In [102]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd
import cv2
import argparse
import mediapipe as mdp
import copy
import os
import csv

In [2]:
# GETS THE REQUIRED ARGUMENTS IN CLI 

# def get_args():
#     parser=argparse.ArgumentParser()

#     parser.add_argument("--device", type= int,default=0)
#     parser.add_argument("--staticimage", action="store_true")
#     parser.add_argument("--width",help="Maximun width", type= int,default=960)
#     parser.add_argument("--height",help="Maximun height", type= int,default=540)

#     parser.add_argument("--minDetection" ,help="Minimum confidence for detection",type=float,default=0.6)
#     parser.add_argument("--minTracking" ,help="Minimum confidenxce for tracking",type=float,default=0.8)

#     args = parser.parse_args()

#     return args


In [3]:
def load_models(use_static_image_mode,min_detection,min_tracking):

#LOAD MODEL TO DETECT LANDMARKS IN HANDS
    mdp_hands=mdp.solutions.hands
    hands=mdp_hands.Hands(
        static_image_mode=use_static_image_mode,
        max_num_hands=2,
        min_detection_confidence=min_detection,
        min_tracking_confidence=min_tracking
        )
    return hands

In [4]:
def camera_input(cam_device,cam_width,cam_height):

#POPERLY INITIALIZE CAMERA FOR WORKING
    cam=cv2.VideoCapture(cam_device)
    cam.set(cv2.CAP_PROP_FRAME_WIDTH,cam_width)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT,cam_height)
    print(cam)
    cam.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [101]:
# A FUNCTION OT SELECT MODE AND TO SET THE NUMBER TO A VALUE FOR TRAINING
# Probaqbly wont be used but whatever

def select_mode(key,number):
    number=-1
    if 48<=key<=57:
        number=key-48
    if key==110:
        mode=0
    if key==107:
        mode=1
    return number,mode


In [57]:
# NOW TO FIND THE EXACT COORDINATES IN THE SCREEN AS THE IMAGES ARE NORMALIZED RIGHT NPW

def screen_coordinates(normalized_coodr,image):
    img_width,img_height=image.shape[0],image.shape[1]

    screen_coordinate=[]

    for i,landmark in enumerate(normalized_coodr[0].landmark):
        screen_x=min(int(landmark.x*img_width),img_width-1)
        screen_y=min(int(landmark.y*img_height),img_height-1)
        #NO NEED FOR LANDMARK Z AND ALSO IMAGE.SHAPE[2] PROVIDES COLOR INDEXES LIKE RGB SO 3
        screen_coordinate.append([screen_x,screen_y])

    return screen_coordinate

In [95]:
#NORMALIZATION SO THAT WE CAN DO SOMETHING EVEN WHEN THE HAND IS MOVED AND IT STILL WORKS ON THE HAND GEUSTURE
#THE NORMALIZED POINTS MAKES IT SO THA WE NEED LESSER DATA AND SIMPLER MODEL TO TRAIN AND DETECT

def normalized_values(screen_coordinate):

    temp_landmark=copy.deepcopy(screen_coordinate)
    base_x,base_y=0,0

    print(type(temp_landmark))

    for i, pixel_val in enumerate(temp_landmark):

        if i==0:
            base_x,base_y=screen_coordinate[0][0],screen_coordinate[0][1]
        
        temp_landmark[i][0]=pixel_val[0]-base_x
        temp_landmark[i][1]=pixel_val[1]-base_y

    temp_landmark=np.array(temp_landmark).flatten()
    max_val=max(list(map(abs,temp_landmark)))

    def normalize_(val):
        return val/max_val
    
    temp_landmark=list(map(normalize_,temp_landmark))

    return temp_landmark
    

        

<class 'list'>


[np.float64(0.0),
 np.float64(0.0),
 np.float64(-0.05154639175257732),
 np.float64(-0.39690721649484534),
 np.float64(-0.14432989690721648),
 np.float64(-0.6649484536082474),
 np.float64(-0.23711340206185566),
 np.float64(-0.8402061855670103),
 np.float64(-0.29381443298969073),
 np.float64(-1.0),
 np.float64(-0.35051546391752575),
 np.float64(-0.3402061855670103),
 np.float64(-0.4742268041237113),
 np.float64(-0.41237113402061853),
 np.float64(-0.5515463917525774),
 np.float64(-0.42783505154639173),
 np.float64(-0.6185567010309279),
 np.float64(-0.4329896907216495),
 np.float64(-0.36597938144329895),
 np.float64(-0.10309278350515463),
 np.float64(-0.5103092783505154),
 np.float64(-0.11855670103092783),
 np.float64(-0.5979381443298969),
 np.float64(-0.10824742268041238),
 np.float64(-0.6804123711340206),
 np.float64(-0.09278350515463918),
 np.float64(-0.35051546391752575),
 np.float64(0.12371134020618557),
 np.float64(-0.4845360824742268),
 np.float64(0.15979381443298968),
 np.float64(-

In [119]:
# NOW TO STORE THE DATAS IN A CSV FILE 
def store_data(number,normalized_one):
    csv_path="/datas/pre_processed.csv"
    
    with open(csv_path,'a',newline="") as send:
        store=csv.writer(send)
        store.writerow([number,*normalized_one])
    
    return


In [125]:
curr_path

'c:\\Users\\sinju\\Documents\\HandsDetectSign\\Hand_gesture_detection'

In [171]:
def image_path(path=''):
    paths=[]
    if not path:
        path=r"datas\photos_hands"
    all_files=list(map(int,os.listdir(path)))

    for i in range(len(all_files)):
        paths.append(os.path.join(path,str(all_files[i])))
    return paths


In [172]:
image_path()


['datas\\photos_hands\\1', 'datas\\photos_hands\\2']

In [ ]:
def draw_landmarks(image, landmark_point):
    if len(landmark_point) > 0:
        # Thumb
        cv2.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]),
                (255, 255, 255), 2)

        # Index finger
        cv2.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]),
                (255, 255, 255), 2)

        # Middle finger
        cv2.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]),
                (255, 255, 255), 2)

        # Ring finger
        cv2.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]),
                (255, 255, 255), 2)

        # Little finger
        cv2.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]),
                (255, 255, 255), 2)

        # Palm
        cv2.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]),
                (255, 255, 255), 2)
        cv2.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]),
                (0, 0, 0), 6)
        cv2.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]),
                (255, 255, 255), 2)


In [116]:
def main():
    
# GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    cam_device=0
    cam_width=960
    cam_height=540
# GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    use_static_image_mode=True
    min_detection=0.6
    min_tracking=0.8

    cam=camera_input(cam_device,cam_width,cam_height)
    hands=load_models(use_static_image_mode,min_detection,min_tracking)

    image=cv2.imread(r"C:\Users\sinju\Documents\HandsDetectSign\photos_hands\image.png")

    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    results=hands.process(image)

    screen_coordinate=screen_coordinates(results.multi_hand_landmarks,image)
    normalized_coodr=normalized_values(screen_coordinate)
    store_data(1,normalized_coodr)


    

main()



< cv2.VideoCapture 000001D183E35F70>
(425, 700, 3)
<class 'list'>


c:\Users\sinju\Documents\HandsDetectSign\handson\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [114]:
csv_path="../datas/pre_processed.csv"
folder_path="../datas/photos_hands"
all_num=list(map(int,(os.listdir(folder_path))))

all_num

[1, 2]

In [46]:
x=results.multi_hand_landmarks
for i,val in enumerate(x[0].landmark):
    print(val.z)

3.2541322525503347e-07
-0.02299955114722252
-0.030182238668203354
-0.03707556799054146
-0.04414438456296921
-0.006494410336017609
-0.019150547683238983
-0.032826147973537445
-0.04419930651783943
-0.007790715899318457
-0.01980663277208805
-0.03442426025867462
-0.04578622430562973
-0.01307669933885336
-0.028383594006299973
-0.042454879730939865
-0.05228200554847717
-0.02082028239965439
-0.036992985755205154
-0.045942820608615875
-0.05158940702676773


In [36]:
# Example function to process hand landmarks
def process_hand_landmarks(hand_landmarks: landmark_pb2.NormalizedLandmarkList):
    for idx, landmark in enumerate(hand_landmarks.landmark):
        print(f'Landmark {idx}: x={landmark.x}, y={landmark.y}, z={landmark.z}')

# Example usage with dummy data
example_landmarks = landmark_pb2.NormalizedLandmarkList()
example_landmarks.landmark.add(x=0.5, y=0.5, z=0.5)
example_landmarks.landmark.add(x=0.6, y=0.6, z=0.6)

process_hand_landmarks(example_landmarks)

0
1


In [15]:
results=main()
output_dict = results.__dict__

# Print the entire dictionary
for key, value in output_dict.items():
    print(f"{key}: {value}")

< cv2.VideoCapture 000001D1AB1DC770>
__doc__: SolutionOutputs(multi_hand_landmarks, multi_hand_world_landmarks, multi_handedness)
__slots__: ()
_fields: ('multi_hand_landmarks', 'multi_hand_world_landmarks', 'multi_handedness')
_field_defaults: {}
__new__: <staticmethod(<function SolutionOutputs.__new__ at 0x000001D1AB3F6700>)>
_make: <classmethod(<function SolutionOutputs._make at 0x000001D1AB1C3D80>)>
_replace: <function SolutionOutputs._replace at 0x000001D1AB1C37E0>
__repr__: <function SolutionOutputs.__repr__ at 0x000001D190311620>
_asdict: <function SolutionOutputs._asdict at 0x000001D190313F60>
__getnewargs__: <function SolutionOutputs.__getnewargs__ at 0x000001D1AB1F9260>
__match_args__: ('multi_hand_landmarks', 'multi_hand_world_landmarks', 'multi_handedness')
multi_hand_landmarks: [landmark {
  x: 0.603820086
  y: 0.531883359
  z: 3.25413225e-007
}
landmark {
  x: 0.579289913
  y: 0.422052443
  z: -0.0229995511
}
landmark {
  x: 0.537265062
  y: 0.348211676
  z: -0.0301822387